# Load gene_info_table

We need to update this table based on existing databases

In [3]:
import pandas as pd

pd.set_option("display.max_rows", 1000)
import sys
from pathlib import Path

repo_dir = Path.cwd().parent.absolute()
sys.path.append(str(repo_dir))

In [4]:
from src.utils import setup_data_dir
from pathlib import Path

setup_data_dir()
data_dir = repo_dir / "data"

File already exists at /Users/rj/personal/GenePT-tools/data/GenePT_emebdding_v2.zip
Extracting files...
Extracting GenePT_emebdding_v2/
Skipping GenePT_emebdding_v2/NCBI_UniProt_summary_of_genes.json - already exists with same size
Skipping GenePT_emebdding_v2/GenePT_gene_embedding_ada_text.pickle - already exists with same size
Skipping GenePT_emebdding_v2/GenePT_gene_protein_embedding_model_3_text.pickle. - already exists with same size
Skipping GenePT_emebdding_v2/NCBI_summary_of_genes.json - already exists with same size
Extraction complete!
Setup finished!


In [5]:
import pandas as pd

gene_info_table = pd.read_parquet(data_dir / "gene_info_table.parquet")
gene_info_table

,ensembl_id,gene_type
index,,
TSPAN6,ENSG00000000003,protein_coding
TNMD,ENSG00000000005,protein_coding
DPM1,ENSG00000000419,protein_coding
SCYL3,ENSG00000000457,protein_coding
C1orf112,ENSG00000000460,protein_coding
...,...,...
LINC02481,ENSG00000246526,None
LINC01856,ENSG00000237574,None
LINC02698,ENSG00000256717,None


In [6]:
from src.utils import setup_data_dir
from pathlib import Path


embedding_dir = data_dir / "GenePT_emebdding_v2"
ncbi_summary_of_genes_path = embedding_dir / "NCBI_summary_of_genes.json"
ncbi_uniprot_summary_of_genes_path = (
    embedding_dir / "NCBI_UniProt_summary_of_genes.json"
)

print("embedding_dir exists:", embedding_dir.exists())
print("ncbi_summary_of_genes_path exists:", ncbi_summary_of_genes_path.exists())
print(
    "ncbi_uniprot_summary_of_genes_path exists:",
    ncbi_uniprot_summary_of_genes_path.exists(),
)

embedding_dir exists: True
ncbi_summary_of_genes_path exists: True
ncbi_uniprot_summary_of_genes_path exists: True


In [7]:
ncbi_summary_of_genes = json.load(open(ncbi_summary_of_genes_path))
ncbi_uniprot_summary_of_genes = json.load(open(ncbi_uniprot_summary_of_genes_path))

{'RPL23AP66',
 'SMIM29',
 'C8orf88',
 'DHX33-DT',
 'FGF7P7',
 'TMEM271',
 'SEPTIN7P12',
 'RAB29',
 'SEPTIN1',
 'RHOG2P',
 'MESD',
 'LRIG2-DT',
 'TIMM29',
 'CNTNAP2-AS1',
 'C1GALT1P3',
 'NECTIN1-AS1',
 'MRGPRX12P',
 'DDX19A-DT',
 'RNY3P15',
 'FLNC-AS1',
 'LINC02872',
 'IQANK1',
 'MRPS18CP7',
 'INKA1',
 'TIMM23B-AGAP6',
 'LINC02634',
 'MAN1B1-DT',
 'MUCL3',
 'MYG1P1',
 'CFAP251',
 'RMC1',
 'TP73-AS2',
 'MIOXP1',
 'LINC02573',
 'DTD1-AS1',
 'PIK3CDP1',
 'VPS35L',
 'LINC01527',
 'GGTA1',
 'MRPL58',
 'PRR32',
 'LINC02110',
 'LINC02660',
 'ADGRE1',
 'IMMP1LP2',
 'ETDA',
 'SNORD38C',
 'NEMP1',
 'LINC01986',
 'SNORA31B',
 'PRR23D2',
 'HMGB1P33',
 'LINC02731',
 'LINC01409',
 'SCAT1',
 'H2AC7',
 'NOL4L-DT',
 'PSMC2P1',
 'ATF7-NPFF',
 'S100A11P5',
 'GPAT4',
 'FAM27E5',
 'LINC01664',
 'MTCYBP33',
 'HPAT5',
 'VNN3P',
 'DNAJB6P7',
 'PPIAP85',
 'GDF5-AS1',
 'DNAJB6P3',
 'LINC01259',
 'ATP5MK',
 'LINC01924',
 'MTARC1',
 'ELOCP6',
 'RPL7AP53',
 'SNORA80D',
 'CDK6-AS1',
 'SPACA9',
 'LINC01582',
 'CDC42-

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

prompt_template = """Tell me about the {0} gene.

Here is the NCBI and UniProt summary of the gene:

{1}

----

In addition to the provided information, please:

1. List any other genes that the gene is associated with, particularly those not mentioned in the summaries above.
2. Highlight any known details related to aging or age-related diseases that might be associated with this gene.
3. List any drug or drug classes that are known to interact with this gene. 
4. Pathways and biological processes that this gene is involved in.
"""

print(
    prompt_template.format(
        "LOC124907803", ncbi_uniprot_summary_of_genes["LOC124907803"]
    )
)

Tell me about the LOC124907803 gene.

Here is the NCBI and UniProt summary of the gene:

Gene Symbol LOC124907803

----

In addition to the provided information, please:

1. List any other genes that the gene is associated with, particularly those not mentioned in the summaries above.
2. Highlight any known details related to aging or age-related diseases that might be associated with this gene.
3. List any drug or drug classes that are known to interact with this gene. 
4. Pathways and biological processes that this gene is involved in.



In [11]:
from openai import OpenAI  # New import

client = OpenAI()  # Initialize client
gene_completion_test = ["LOC124907803"]
for gene in sorted(gene_completion_test):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt_template.format(
                    gene, ncbi_uniprot_summary_of_genes[gene]
                ),
            }
        ],
        temperature=0.0,
    )
    print(
        f"""
{ncbi_uniprot_summary_of_genes[gene]}
                    
{completion.choices[0].message.content}

"""
    )
print("-" * 100)


Gene Symbol LOC124907803
                    
The LOC124907803 gene is a gene of interest, but specific details about it may not be as well characterized as more prominent genes. Here’s a summary based on your request:

### 1. Associated Genes
While specific associations for LOC124907803 may not be well-documented, genes often have functional relationships or co-expression patterns with other genes. To find associated genes, one would typically look at gene co-expression databases or perform network analyses. However, without specific data from NCBI or other databases, I cannot provide a definitive list of associated genes.

### 2. Aging and Age-Related Diseases
As of my last update, there may not be direct studies linking LOC124907803 to aging or age-related diseases. However, many genes can have indirect roles in these processes through their involvement in cellular functions, stress responses, or metabolic pathways. To find specific information, literature searches in databases lik

In [45]:
import json


def write_batch_requests_jsonl(requests: list, filename: str) -> Path:
    """
    Write a list of JSON objects to a JSONL file in the batch-requests directory.

    Args:
        requests: List of JSON objects to write
        filename: Name of the file (without .jsonl extension)

    Returns:
        Path: Path object pointing to the created file
    """
    # Create the directory path

    output_dir = data_dir / "generated/batch-requests"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Ensure filename has .jsonl extension
    if not filename.endswith(".jsonl"):
        filename += ".jsonl"

    output_path = output_dir / filename

    # Write the requests line by line
    with open(output_path, "w") as f:
        for request in requests:
            json_line = json.dumps(request)
            f.write(json_line + "\n")

    return output_path

In [72]:
small_batch_of_requests = [
    {
        "custom_id": f"initial-100-request-{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a detail oriented bioinformatics and genetics expert help to give accurate and precise descriptions of genes",
                },
                {
                    "role": "user",
                    "content": prompt_template.format(
                        gene, ncbi_uniprot_summary_of_genes[gene]
                    ),
                },
            ],
            "max_tokens": 2000,
        },
    }
    for i, gene in enumerate(list(ncbi_uniprot_summary_of_genes.keys())[:100])
]

full_batch_of_requests = [
    {
        "custom_id": f"initial-100-request-{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a detail oriented bioinformatics and genetics expert help to give accurate and precise descriptions of genes",
                },
                {
                    "role": "user",
                    "content": prompt_template.format(
                        gene, ncbi_uniprot_summary_of_genes[gene]
                    ),
                },
            ],
            "max_tokens": 2000,
        },
    }
    for i, gene in enumerate(list(ncbi_uniprot_summary_of_genes.keys()))
]

"""
{
    "input": "The food was delicious and the waiter...",
    "model": "text-embedding-ada-002",
    "encoding_format": "float"
  }
  """

In [73]:
path_to_batch_requests_jsonl = write_batch_requests_jsonl(
    full_batch_of_requests, "full_batch_of_requests.jsonl"
)

batch_input_file = client.files.create(
    file=open(path_to_batch_requests_jsonl, "rb"), purpose="batch"
)

print(batch_input_file)

FileObject(id='file-FAVFkhiXMevKedjPSyDvCb', bytes=48069520, created_at=1736127901, filename='full_batch_of_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [74]:
batch_input_file_id = batch_input_file.id
batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={"description": "small batch for testing the batch API"},
)

print(batch)

Batch(id='batch_677b35ab12cc81909d51998611a5bc14', completion_window='24h', created_at=1736127915, endpoint='/v1/chat/completions', input_file_id='file-FAVFkhiXMevKedjPSyDvCb', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736214315, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'small batch for testing the batch API'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [166]:
def monitor_batch_status(client, batch, check_interval=60, verbose=True):
    """
    Monitor the status of an OpenAI batch processing job.

    Args:
        client: OpenAI client instance
        batch: Batch object returned from client.batches.create()
        check_interval: Time in seconds between status checks (default: 60)
        verbose: Whether to print progress updates (default: True)

    Returns:
        BatchStatus object for the completed batch
    """
    try:
        # Monitor completion progress
        while True:
            batch_status = client.batches.retrieve(batch.id)

            completed = batch_status.request_counts.completed
            failed = batch_status.request_counts.failed
            total = batch_status.request_counts.total

            if verbose:
                print(
                    f"{time.strftime('%Y-%m-%d %H:%M:%S')} - "
                    f"Completed: {completed}, Failed: {failed}, Total: {total}"
                )

            if completed + failed == total:
                break

            time.sleep(check_interval)

        # Wait for final batch completion
        while True:
            batch_status = client.batches.retrieve(batch.id)
            if (
                batch_status.completed_at is not None
                or batch_status.failed_at is not None
            ):
                break
            if verbose:
                print(".", end="", flush=True)
            time.sleep(check_interval)

        if verbose:
            print("\nBatch completed")

        return batch_status

    except Exception as e:
        print(f"Error monitoring batch status: {str(e)}")
        raise

In [76]:
# import time

# while True:
#     batch_status = client.batches.retrieve(batch.id)

#     completed = batch_status.request_counts.completed
#     failed = batch_status.request_counts.failed
#     total = batch_status.request_counts.total

#     print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - Completed: {completed}, Failed: {failed}, Total: {total}")

#     if completed + failed == total:
#         break

#     time.sleep(60)

# while True:
#     batch_status = client.batches.retrieve(batch.id)
#     if batch_status.completed_at is not None or batch_status.failed_at is not None:
#         break
#     print('.', end='', flush=True)
#     time.sleep(60)

# print()
# print("Batch completed")

2025-01-05 19:26:57 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:27:57 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:28:57 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:29:58 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:30:58 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:31:58 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:32:58 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:33:58 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:34:59 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:35:59 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:36:59 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:37:59 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:38:59 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:40:00 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:41:00 - Completed: 32686, Failed: 0, Total: 33703
2025-01-05 19:42:00 - Completed: 33703, 

In [ ]:
batch_status = monitor_batch_status(client, batch, check_interval=60, verbose=True)

In [77]:
batch_status

Batch(id='batch_677b35ab12cc81909d51998611a5bc14', completion_window='24h', created_at=1736127915, endpoint='/v1/chat/completions', input_file_id='file-FAVFkhiXMevKedjPSyDvCb', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736137398, error_file_id=None, errors=None, expired_at=None, expires_at=1736214315, failed_at=None, finalizing_at=1736134891, in_progress_at=1736127923, metadata={'description': 'small batch for testing the batch API'}, output_file_id='file-E96uEVPiVeRi72Z8QKL6pq', request_counts=BatchRequestCounts(completed=33703, failed=0, total=33703))

In [169]:
def save_batch_response(client, batch_status, data_dir, filename) -> Path:
    """
    Save the response from an OpenAI batch operation to a JSONL file.

    Args:
        client: OpenAI client instance
        batch_status: BatchStatus object from completed batch
        data_dir: Path object pointing to base data directory
        filename: Name of the output file

    Returns:
        Path: Path object pointing to the saved response file
    """
    # Get the response content
    file_response = client.files.content(batch_status.output_file_id)

    # Create output path
    output_path = data_dir / "generated" / "batch-requests" / filename
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Write response to file
    with open(output_path, "w") as f:
        f.write(file_response.text)

    return output_path

In [78]:
file_response = client.files.content(batch_status.output_file_id)
full_batch_response_path = (
    data_dir / "generated" / "batch-requests" / "full_batch_response.jsonl"
)
with open(full_batch_response_path, "w") as f:
    f.write(file_response.text)

full_batch_response_path

PosixPath('data/generated/batch-requests/full_batch_response.jsonl')

In [79]:
import json

responses = []
with open(full_batch_response_path, "r") as f:
    for line in f:
        responses.append(json.loads(line))

In [80]:
responses[:3]

[{'id': 'batch_req_677b50ebb9c48190b12561a413e39fce',
  'custom_id': 'initial-100-request-0',
  'response': {'status_code': 200,
   'request_id': '9259822327f048c92f14a56dced39019',
   'body': {'id': 'chatcmpl-AmWTRZqWaZuu51Low98VpwJvqNxll',
    'object': 'chat.completion',
    'created': 1736127937,
    'model': 'gpt-4o-mini-2024-07-18',
    'choices': [{'index': 0,
      'message': {'role': 'assistant',
       'content': "### Overview of LINC01409 Gene\n\n**Gene Symbol:** LINC01409  \n**Gene Type:** Long Intergenic Non-Coding RNA  \n**Location:** Human Chromosome (specific location may vary by genome assembly)  \n**Function:** As a long non-coding RNA (lncRNA), LINC01409 is not translated into protein but may play roles in gene regulation, chromatin remodeling, and cellular processes by acting as a molecular scaffold or modulator.\n\n### Associated Genes\n\nLINC01409 may be associated with various genes. While specific associations can vary based on research findings, lncRNAs like LI

In [128]:
# Create lists to store our data
gene_names = list(ncbi_uniprot_summary_of_genes.keys())
descriptions = list(ncbi_uniprot_summary_of_genes.values())
gpt_responses = [
    response["response"]["body"]["choices"][0]["message"]["content"]
    for response in responses
]

# Create the DataFrame
gene_descriptions_df = pd.DataFrame(
    {"description": descriptions, "gpt_response": gpt_responses}, index=gene_names
)

# Rename the index
gene_descriptions_df.index.name = "gene_name"

# Display the first few rows
gene_descriptions_df

,description,gpt_response
gene_name,,
LINC01409,Gene Symbol LINC01409,### Overview of LINC01409 Gene\n\n**Gene Symbo...
FAM87B,Gene Symbol FAM87B,"The **FAM87B** gene, designated as *family wit..."
LINC01128,Gene Symbol LINC01128,**Gene Overview: LINC01128**\n\nLINC01128 (Lon...
LINC00115,Gene Symbol LINC00115,### LINC00115 Gene Summary\n\n**Gene Symbol**:...
FAM41C,Gene Symbol FAM41C,### FAM41C Gene Overview\n\n**Gene Symbol:** F...
...,...,...
ZUP1,Gene Symbol ZUP1 This gene encodes a protein c...,### ZUP1 Gene Overview\n\n**Gene Symbol:** ZUP...
ZWILCH,Gene Symbol ZWILCH Involved in protein localiz...,The ZWILCH gene encodes a protein that plays a...
ZXDA,Gene Symbol ZXDA This gene encodes one of two ...,The **ZXDA** gene encodes a protein that has b...


In [87]:
gene_descriptions_df.loc["ZUP1"]

description     Gene Symbol ZUP1 This gene encodes a protein c...
gpt_response    ### ZUP1 Gene Overview\n\n**Gene Symbol:** ZUP...
Name: ZUP1, dtype: object

In [142]:
descriptions_with_ensemble_ids = gene_descriptions_df.merge(
    gene_info_table, left_index=True, right_index=True, how="left"
)

In [224]:
print(gene_descriptions_df.shape)
print(gene_info_table.shape)
print(descriptions_with_ensemble_ids.shape)
print(gene_info_table.loc["SNORD112"].shape)

(33703, 2)
(84425, 2)
(37262, 4)
(51, 2)


In [143]:
descriptions_with_ensemble_ids

,description,gpt_response,ensembl_id,gene_type
gene_name,,,,
LINC01409,Gene Symbol LINC01409,### Overview of LINC01409 Gene\n\n**Gene Symbo...,ENSG00000237491,None
FAM87B,Gene Symbol FAM87B,"The **FAM87B** gene, designated as *family wit...",ENSG00000177757,lincRNA
LINC01128,Gene Symbol LINC01128,**Gene Overview: LINC01128**\n\nLINC01128 (Lon...,ENSG00000228794,lncRNA
LINC00115,Gene Symbol LINC00115,### LINC00115 Gene Summary\n\n**Gene Symbol**:...,ENSG00000225880,lincRNA
FAM41C,Gene Symbol FAM41C,### FAM41C Gene Overview\n\n**Gene Symbol:** F...,ENSG00000230368,lincRNA
...,...,...,...,...
ZUP1,Gene Symbol ZUP1 This gene encodes a protein c...,### ZUP1 Gene Overview\n\n**Gene Symbol:** ZUP...,ENSG00000153975,None
ZWILCH,Gene Symbol ZWILCH Involved in protein localiz...,The ZWILCH gene encodes a protein that plays a...,ENSG00000174442,protein_coding
ZXDA,Gene Symbol ZXDA This gene encodes one of two ...,The **ZXDA** gene encodes a protein that has b...,ENSG00000198205,protein_coding


In [144]:
missing_ensembl_ids = descriptions_with_ensemble_ids[
    descriptions_with_ensemble_ids.ensembl_id.isna()
].index
missing_ensembl_ids

descriptions_with_ensemble_ids.loc[missing_ensembl_ids]

,description,gpt_response,ensembl_id,gene_type
gene_name,,,,
TNFRSF14-AS1,Gene Symbol TNFRSF14-AS1,### Overview of TNFRSF14-AS1 Gene\n\n**Gene Sy...,None,None
LINC01672,Gene Symbol LINC01672,### LINC01672 Gene Overview\n\n**Gene Symbol:*...,None,None
LINC01714,Gene Symbol LINC01714,### LINC01714 Gene Overview\n\n**Gene Symbol:*...,NaN,NaN
KAZN-AS1,Gene Symbol KAZN-AS1,### KAZN-AS1 Gene Overview\n\n**Gene Symbol:**...,None,None
ZNF436-AS1,Gene Symbol ZNF436-AS1 Protein summary:,### ZNF436-AS1 Gene Overview\n\n**Gene Symbol:...,None,None
...,...,...,...,...
ZKSCAN2-DT,Gene Symbol ZKSCAN2-DT,### ZKSCAN2-DT Gene Overview\n\n**Gene Symbol:...,None,None
ZNF561-AS1,Gene Symbol ZNF561-AS1 Protein summary:,### Overview of ZNF561-AS1 Gene\n\n**Gene Symb...,None,None
ZNF602P,Gene Symbol ZNF602P,"The ZNF602P gene is a pseudogene, specifically...",None,None


In [146]:
embedding_prompts = {
    gene: prompt_template.format(
        gene,
        ncbi_uniprot_summary_of_genes[gene],
    )
}

### LINC01714 Gene Overview

**Gene Symbol:** LINC01714

LINC01714 is categorized as a long intergenic non-coding RNA (lncRNA) gene, located on chromosome 1. Non-coding RNAs, particularly lncRNAs, play critical roles in various biological processes, including gene regulation and cellular differentiation.

### 1. Associated Genes
Specific associations of LINC01714 with other genes are often revealed through expression studies and co-expression networks. However, general associations might include:
- **KRTAPs** (Keratin-Associated Proteins): LINC01714 is implicated in keratinocyte function.
- **GRB2**: Involved in signaling pathways that may be influenced by lncRNAs, including LINC01714.

LINC01714 is often examined in studies focused on skin, cancer, and developmental processes. However, specific databases such as STRING or BioGRID should be checked for detailed and updated interaction data with additional genes.

### 2. Aging and Age-Related Diseases
Research regarding LINC01714's role

In [195]:
embedding_prompt_template = """
{0}

{1}
"""

full_batch_of_embedding_requests = [
    {
        "custom_id": f"full-batch-embedding-request-{i}",
        "method": "POST",
        "url": "/v1/embeddings",
        "body": {
            "model": "text-embedding-3-large",
            "input": embedding_prompt_template.format(
                row.description, row.gpt_response
            ),
            "encoding_format": "float",
        },
    }
    for i, (gene_name, row) in enumerate(descriptions_with_ensemble_ids.iterrows())
    # oops, this should have been:
    # for i, (gene_name, row) in enumerate(gene_descriptions_df.iterrows())
    # I got a bunch of duplicates, no big deal, we will just average the embeddings
]

In [196]:
path_to_embedding_batch_requests_jsonl = write_batch_requests_jsonl(
    full_batch_of_embedding_requests, "full_batch_of_embedding_requests.jsonl"
)

embedding_batch_input_file = client.files.create(
    file=open(path_to_embedding_batch_requests_jsonl, "rb"), purpose="batch"
)

print(embedding_batch_input_file)

FileObject(id='file-HqYVsaFqcoujYJGdkCZRPZ', bytes=148554242, created_at=1736308012, filename='full_batch_of_embedding_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [197]:
embedding_batch_input_file_id = embedding_batch_input_file.id
embedding_batch = client.batches.create(
    input_file_id=embedding_batch_input_file_id,
    endpoint="/v1/embeddings",
    completion_window="24h",
    metadata={"description": "full batch for testing the batch embedding API"},
)

print(embedding_batch)

Batch(id='batch_677df53dad408190a583ca4164f4f5b7', completion_window='24h', created_at=1736308029, endpoint='/v1/embeddings', input_file_id='file-HqYVsaFqcoujYJGdkCZRPZ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736394429, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'full batch for testing the batch embedding API'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [209]:
final_status = monitor_batch_status(
    client, embedding_batch, check_interval=60, verbose=True
)

2025-01-07 19:54:44 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 19:55:44 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 19:56:44 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 19:57:44 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 19:58:48 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 19:59:48 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 20:00:48 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 20:01:49 - Completed: 0, Failed: 0, Total: 37262
2025-01-07 20:02:49 - Completed: 367, Failed: 0, Total: 37262
2025-01-07 20:03:49 - Completed: 802, Failed: 0, Total: 37262
2025-01-07 20:04:49 - Completed: 1135, Failed: 0, Total: 37262
2025-01-07 20:05:50 - Completed: 1479, Failed: 0, Total: 37262
2025-01-07 20:06:50 - Completed: 2203, Failed: 0, Total: 37262
2025-01-07 20:07:51 - Completed: 2630, Failed: 0, Total: 37262
2025-01-07 20:08:51 - Completed: 3454, Failed: 0, Total: 37262
2025-01-07 20:09:52 - Completed: 4292, Failed: 0, Total: 37262
2025-01-07 20:10:5

In [210]:
full_embedding_batch_response_path = save_batch_response(
    client, final_status, data_dir, "full_embedding_batch_response.jsonl"
)

In [211]:
responses = []
with open(full_embedding_batch_response_path, "r") as f:
    for line in f:
        responses.append(json.loads(line))

In [213]:
embedding_df = pd.DataFrame(
    (response["response"]["body"]["data"][0]["embedding"] for response in responses),
    index=pd.Series(descriptions_with_ensemble_ids.index),
)
embedding_df

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
gene_name,,,,,,,,,,,,,,,,,,,,,
LINC01409,-0.030524,0.025640,-0.013194,-0.007818,0.005398,-0.012957,0.051630,-0.012808,-0.004043,0.033340,...,-0.013343,0.010135,0.028207,-0.013306,0.013630,-0.007307,0.027484,-0.012783,-0.001903,-0.003386
FAM87B,-0.024839,0.017146,-0.008555,-0.001628,0.005903,-0.045881,0.020695,-0.037419,0.033846,0.015881,...,-0.003403,0.006352,0.021501,-0.003828,-0.014020,-0.021228,0.030869,-0.022767,-0.021588,-0.015620
LINC01128,-0.026745,0.003518,-0.012758,0.009890,-0.005350,-0.017632,0.023561,-0.006434,0.018991,-0.003447,...,-0.009974,0.005518,0.022163,-0.026435,0.008667,-0.018667,0.027263,-0.018085,-0.006084,-0.041840
LINC00115,-0.029675,0.000203,-0.008749,-0.015585,0.007906,-0.020653,0.059146,-0.016168,0.012367,0.003602,...,-0.016941,0.010909,0.023719,-0.018436,0.006098,-0.022351,0.025443,-0.018309,-0.004365,-0.030435
FAM41C,-0.026250,0.040750,-0.012944,0.000955,-0.023294,0.007636,0.005015,-0.020785,-0.000150,0.013318,...,-0.026708,-0.000848,0.028349,-0.011514,-0.002913,-0.018795,0.003224,-0.005881,-0.033078,-0.002396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZUP1,-0.003096,-0.001112,-0.006090,0.015157,0.014000,-0.025319,0.022479,-0.015749,0.072091,-0.011509,...,0.009268,0.013093,0.007270,-0.012725,0.022913,0.001146,0.012133,0.006524,-0.006622,-0.008571
ZWILCH,0.026960,0.015145,-0.005318,0.020834,-0.020648,-0.023579,0.006989,-0.033366,0.034188,0.020290,...,-0.001021,0.022094,0.041986,-0.004635,0.014667,0.000080,-0.000277,0.012685,-0.010888,-0.011962
ZXDA,0.001598,0.023380,-0.008155,0.037740,0.036181,-0.024497,0.033452,-0.031684,0.056246,-0.015036,...,-0.010013,0.005367,0.016297,-0.008200,0.017532,-0.013516,0.012327,0.014997,-0.006573,-0.015894


In [215]:
pd.Series(descriptions_with_ensemble_ids.index).value_counts()

gene_name
SNORD112    51
SNORA31     26
SNORA40     24
SNORA48     21
SNORA25     20
            ..
EMX2OS       1
EMX2         1
EMSY         1
EMP2         1
ZYXP1        1
Name: count, Length: 33703, dtype: int64

In [228]:
embedding_df_averaged = embedding_df.groupby(level=0).mean()

In [229]:
embedding_df_averaged.loc["SNORA31"]

0      -0.014382
1      -0.005982
2      -0.012120
3       0.005852
4       0.001855
          ...   
3067   -0.023921
3068    0.015496
3069    0.010181
3070    0.005584
3071    0.005180
Name: SNORA31, Length: 3072, dtype: float64

In [230]:
embedding_df.loc["SNORA31"].describe()

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,...,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,-0.014382,-0.005982,-0.012120,0.005852,0.001855,-0.037218,0.037500,-0.009743,0.025648,0.012815,...,-0.001355,0.018275,0.026469,-0.009365,-0.007201,-0.023921,0.015496,0.010181,0.005584,0.005180
std,0.000053,0.000078,0.000042,0.000067,0.000094,0.000064,0.000074,0.000067,0.000058,0.000078,...,0.000046,0.000034,0.000042,0.000039,0.000066,0.000059,0.000030,0.000055,0.000026,0.000047
min,-0.014469,-0.006079,-0.012215,0.005754,0.001637,-0.037326,0.037321,-0.009851,0.025560,0.012599,...,-0.001435,0.018214,0.026410,-0.009415,-0.007356,-0.024037,0.015430,0.010041,0.005514,0.005112
25%,-0.014409,-0.006027,-0.012127,0.005811,0.001820,-0.037273,0.037445,-0.009804,0.025602,0.012796,...,-0.001378,0.018255,0.026439,-0.009400,-0.007244,-0.023957,0.015474,0.010166,0.005581,0.005148
50%,-0.014400,-0.006008,-0.012115,0.005832,0.001879,-0.037211,0.037517,-0.009738,0.025644,0.012835,...,-0.001355,0.018277,0.026453,-0.009372,-0.007179,-0.023928,0.015501,0.010197,0.005593,0.005165
75%,-0.014355,-0.005956,-0.012094,0.005874,0.001924,-0.037184,0.037549,-0.009711,0.025679,0.012855,...,-0.001323,0.018289,0.026507,-0.009345,-0.007148,-0.023873,0.015519,0.010219,0.005599,0.005211
max,-0.014211,-0.005785,-0.012059,0.006050,0.001987,-0.037094,0.037652,-0.009550,0.025793,0.012919,...,-0.001269,0.018349,0.026572,-0.009279,-0.007123,-0.023817,0.015537,0.010236,0.005614,0.005274


In [231]:
embedding_df_averaged.shape

(33703, 3072)

In [234]:
# Create embeddings directory if it doesn't exist
embedding_dir = data_dir / "generated/embeddings"
embedding_dir.mkdir(parents=True, exist_ok=True)

embedding_df_averaged.columns = [str(col) for col in embedding_df_averaged.columns]
# Save the averaged embeddings to a parquet file
embedding_df_averaged.to_parquet(
    embedding_dir / "embedding_associations_age_drugs_pathways_openai_large.parquet"
)